# OSO

In [1]:
import polars as pl
from google.cloud import bigquery

client = bigquery.Client()

In [2]:
QUERY = """
select
  artifact_url as target,
  is_fork,
  star_count,
  fork_count,
  created_at,
  updated_at
from `opensource-observer.oso.repositories_v0`
where artifact_url in ('https://github.com/libp2p/go-libp2p')
"""

query_job = client.query(QUERY)
rows = query_job.result()

oso_df = pl.from_arrow(rows.to_arrow())

print(f"Rows: {oso_df.shape[0]}")

oso_df.head()

Rows: 1


target,is_fork,star_count,fork_count,created_at,updated_at
str,bool,i64,i64,"datetime[μs, UTC]","datetime[μs, UTC]"
"""https://github.com/libp2p/go-l…",false,6140,1088,2015-09-30 23:24:32 UTC,2024-12-19 18:46:49 UTC
